In [ ]:
!pip install -U pip
!pip install --index-url https://download.pytorch.org/whl/cu118 torch torchvision torchaudio
!pip install mediapipe

!pip install -U \
certifi \
charset-normalizer \
cmake \
filelock \
fsspec \
huggingface-hub \
idna \
Jinja2 \
joblib \
kaggle \
lit \
MarkupSafe \
mpmath \
networkx \
numpy \
nvidia-cublas-cu12 \
nvidia-cuda-runtime-cu12 \
nvidia-cudnn-cu12 \
packaging \
pandas \
Pillow \
protobuf \
python-dateutil \
python-slugify \
pytz \
PyYAML \
regex \
requests \
safetensors \
scikit-learn \
scipy \
six \
sympy \
tensorboardX \
text-unidecode \
threadpoolctl \
tokenizers \
torch torchvision torchaudio \
tqdm \
transformers \
triton \
typing_extensions \
tzdata \
urllib3


In [ ]:
import h5py
import numpy as np
import glob

h5_files = glob.glob("./data/*.h5")
all_samples = []
all_labels = []

def infer_label_from_name(name):
    if "Angry" in name or "_an_" in name:
        return 0
    elif "Happy" in name or "_ha_" in name:
        return 1
    elif "Neutral" in name or "_nu_" in name:
        return 2
    elif "Sad" in name or "_sa_" in name:
        return 3
    else:
        return -1  # unknown

for p in h5_files:
    print(f"Processing {p}")
    with h5py.File(p, 'r') as f:
        for k in f.keys():
            data = f[k][()]

            if np.isscalar(data) or data.size == 0:
                print(f"Skipping scalar or empty dataset {p}::{k}")
                continue

            if data.ndim == 1:
                if data.size == 48:
                    data = data.reshape(1, 48)
                else:
                    print(f"Skipping 1D dataset with unexpected length: {p}::{k}, length={data.size}")
                    continue

            if data.ndim != 2 or data.shape[1] != 48:
                print(f"Skipping dataset with unexpected dims {p}::{k} shape={data.shape}")
                continue

            n = data.shape[0]
            lbl = infer_label_from_name(k)
            if lbl == -1:
                print(f"Skipping dataset with unknown label: {p}::{k}")
                continue

            all_samples.append(data.astype('float32'))
            all_labels.append(np.full(n, lbl, dtype='int64'))

if len(all_samples) == 0:
    raise RuntimeError("No data read from H5 files!")

X = np.vstack(all_samples)
y = np.concatenate(all_labels)

print("Final shapes:", X.shape, y.shape)


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import joblib

joblib.dump(clf, "emotion_classifier_rf.pkl")
print("Model saved!")

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import joblib
joblib.dump(scaler, "scaler.pkl")
print("Scaler saved as scaler.pkl")


In [ ]:
import os
print(os.listdir())


In [ ]:
scaler = joblib.load("scaler.pkl")
clf = joblib.load("emotion_classifier_rf.pkl")


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from google.colab.patches import cv2_imshow

model = joblib.load('emotion_classifier_rf.pkl')
scaler = joblib.load('scaler.pkl')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

def extract_features(results):
    features = []
    if results.pose_landmarks:
        for idx, lm in enumerate(results.pose_landmarks.landmark):
            if idx in [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31]:
                features.extend([lm.x, lm.y, lm.z])
    if len(features) != 48:
        return None
    return np.array(features).reshape(1, -1)

video_path = 'testing.mp4'
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    features = extract_features(results)
    if features is None:
        continue
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)[0]

    overlay_text = f'Predicted Emotion: {prediction}'
    cv2.putText(frame, overlay_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1.0, (0, 255, 0), 2)

    cv2_imshow(frame)


cap.release()
pose.close()
print("Video processing done.")
